In [1]:
import pandas as pd
import json

def read_eval(path):
    return pd.read_json(path, orient="index")

In [2]:
def get_number_fixed(df):
    count = 0
    for index, row in df.iterrows():
        for fix in row.fixes:
            if fix["identical"] and row.buggy_line != row.fixed_line:
                count += 1
                break
    return "%d / %d (%.2f%%)" % (count, len(df), count/len(df))

In [3]:
def get_table_entry(lr, epochs, early_stop, beam_width, beam_groups, rp, tbb, tbs, tdb, tds, tb, td):
    return {"Hyper-paramters": {
                "Learning Rate" : {"" : lr},
                "Epochs" : {"" : epochs},
                "Early Stop": {"" : early_stop},
                "Beam Search": {"Beam Width" : beam_width, "Beam Groups" : beam_groups},
                "Repetition Penalty": {"" : rp},
            },
            "Training": {
                "Bears" : {"BugLab" : tbb, "SelfAPR": tbs},
                "Defects4J" : {"BugLab" : tdb, "SelfAPR": tds}
            },
            "Testing":{
                "Bears" : { "" : tb },
                "Defects4J" : { "" : td}
            }
       }

In [4]:
tb = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_buglab_single_line_15epochs/eval_bears_bw_100_bg_1_rp_1.0.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_buglab_single_line_15epochs/eval_defects4j_bw_100_bg_1_rp_1.0.json"))
entries = [get_table_entry(0.0001, 15, "x", 100, 1, 1.0, "", "", "x", "", tb, td)]

tb = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_buglab_single_line_15epochs/eval_bears_bw_100_bg_1_rp_1.0.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_buglab_single_line_15epochs/eval_defects4j_bw_100_bg_1_rp_1.0.json"))
entries += [get_table_entry(0.0001, 15, "x", 100, 1, 1.0, "x", "", "", "", tb, td)]

tb = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_selfapr_single_line_15epochs/eval_bears_bw_100_bg_1_rp_1.0.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_defects4j_selfapr_single_line_15epochs/eval_defects4j_bw_100_bg_1_rp_1.0.json"))
entries += [get_table_entry(0.0001, 15, "x", 100, 1, 1.0, "", "", "", "x", tb, td)]

tb = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_selfapr_single_line_15epochs/eval_bears_bw_100_bg_1_rp_1.0.json"))
td = get_number_fixed(read_eval("./inesc_models/pre_trained_bears_selfapr_single_line_15epochs/eval_defects4j_bw_100_bg_1_rp_1.0.json"))
entries += [get_table_entry(0.0001, 15, "x", 100, 1, 1.0, "", "x", "", "", tb, td)]

In [5]:
table = pd.json_normalize(data=entries)
table.columns = table.columns.str.split(".").map(tuple)
table

Hyper-paramters                                            \
    Learning Rate Epochs Early Stop Beam Search               
                                     Beam Width Beam Groups   
0          0.0001     15          x         100           1   
1          0.0001     15          x         100           1   
2          0.0001     15          x         100           1   
3          0.0001     15          x         100           1   

                     Training                                   Testing  \
  Repetition Penalty    Bears         Defects4J                   Bears   
                       BugLab SelfAPR    BugLab SelfAPR                   
0                1.0                          x          0 / 59 (0.00%)   
1                1.0        x                            0 / 59 (0.00%)   
2                1.0                                  x  0 / 59 (0.00%)   
3                1.0                x                    3 / 59 (0.05%)   

                     
          Defects4J  
                     
0  10 / 428 (0.02%)  
1   8 / 428 (0.02%)  
2  15 / 428 (0.04%)  
3  12 / 428 (0.03%)